In [1]:
import requests
import lxml.html
import pandas as pd
from bs4 import BeautifulSoup
import os

In [2]:
res = requests.get("https://news.naver.com/section/101")
res

<Response [200]>

In [3]:
root = lxml.html.fromstring(res.text)
root

<Element html at 0x2e78bb18680>

In [4]:
urls = []
for link in root.cssselect('a.sa_text_title'):
    urls.append(link.attrib["href"])

In [5]:
print(urls[0])
print(len(urls))

https://n.news.naver.com/mnews/article/214/0001348474
46


In [6]:
articles = []
titles = []
for u in urls:
    if not u.startswith("http"):
        continue

    res = requests.get(u)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    
    # 제목 추출
    title = soup.title
    # get_text()를 이용하여 태그 없이 저장
    title = title.get_text()
    
    # 기사 추출
    article = soup.article
    # get_text()를 이용하여 태그 없이 저장
    body_elements = article.get_text()
    
    titles.append(title)
    articles.append(body_elements)

In [7]:
df = pd.DataFrame({"title":titles , "article":articles})
df.head()

,title,article
0,개미 반란 이끈 '키스 질' 귀환에 '게임스톱' 주가 70% ↑ 폭등,\n\n\n\n\n자료사진 [연합뉴스 제공]대표적인 '밈(Meme) 주식'인 게임스...
1,"지오영, 동물의약품 유통 강화…""안정·신속 공급 최선""",\n맞춤형 콜드체인 유통 시스템 갖춰…동물약 유통 사업 확대\n\n\n\nⓒ New...
2,"우리은행, 제4인터넷전문은행 참여…한국신용데이터 컨소시엄 합류",\n\n\n\n\n우리은행. 연합뉴스우리은행이 한국신용데이터(KCD)가 추진하는 인...
3,"농심, 프랑스 거점 삼아 유럽 본격 공략",\n\n\n\n\n농심 제공 농심이 프랑스 대형유통업체에 대한 판매망 확대를 통해...
4,가처분 결과가 분수령…‘하이브vs민희진’ 운명은,\n[마켓인]가처분 결과 따라 임시주총 결론 달라져인용시 민희진 대표 해임 불가 전...


In [13]:
# 파일명
file_name = "ai_기사2.xlsx"

# 파일이 이미 존재하는지 확인하고 삭제
if os.path.exists(file_name):
    os.remove(file_name)

# Excel 파일 생성
df.to_excel(file_name)